In [22]:
import time
import json
from datetime import timedelta
from bs4 import BeautifulSoup
import asyncio
from aiohttp import ClientSession
from asyncio.locks import Semaphore
from typing import List

In [23]:
TELEGRAM_BASE_URL = 'https://t.me/s/whale_alert_io'
TWEETS_PER_PAGE = 20

In [3]:
def get_transaction_link(tweet):
    return [link.get('href') 
            for link in tweet.find_all('a') 
            if link.get('href').startswith('https://whale-alert.io/transaction')]
    
def scrape_tweets(tweet_num: int, session: ):
    page = requests.get(f'{TELEGRAM_BASE_URL}?before={tweet_num}')
    html = page.content
    soup = BeautifulSoup(html, 'html.parser')
    
    tweets = soup.find_all("div", {"class": "tgme_widget_message_text"})
    data = [
        {
            'text': tweet.text, 
            'links': get_transaction_link(tweet)
        }
        for tweet in tweets
    ]
    
    return data

In [4]:
tweet_num = 32
end_tweet_num = 41839

In [5]:
data = []
while tweet_num < end_tweet_num:
    data += scrape_tweets(tweet_num)
    tweet_num += 20

In [6]:
len(data)

41823

In [11]:
# Remove tweets that weren't transactions
data = [{'text': tweet['text'], 'link': tweet['links'][0]} 
        for tweet in data 
        if len(tweet['links']) == 1 and tweet['text'].count('Details') == 1]

In [12]:
len(data)

41776

In [13]:
with open('../data/tweets_scrape_raw.json', 'w') as f:
    f.write(json.dumps({'data': data}, indent=True))